<a href="https://colab.research.google.com/github/P202U/Job-scrap/blob/main/Job_Scrap_0_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [11]:
keyword = 'fullstack'
location = 'Mumbai'
pageNum = 0
job_url = f'https://www.linkedin.com/jobs/search?keywords={keyword}&location={location}&pageNum={pageNum}'

response = requests.get(job_url)
job_data = response.text
soup = BeautifulSoup(job_data, "lxml")
page_jobs = soup.find_all('div', {'class': 'base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card'})

In [12]:
job_id_list = []

page_jobs = soup.find_all('div', class_='job-search-card')

for job_card in page_jobs:
    try:
        urn_string = job_card.get("data-entity-urn")

        if urn_string:
            job_id = urn_string.split(":")[-1]

            print(job_id)
            job_id_list.append(job_id)

    except (AttributeError, IndexError) as e:
        print(f"Skipping a job card due to an error: {e}")
        continue

print("\nAll extracted Job IDs:", job_id_list)

4290796891
4259114971
4220839450
3849501886
3627622884
4292967130
4260012242
4255094324
4259098167
4259116372
4048230355
4276958556
4273221487
4240914500
4281485208
4275350664
4168111890
4294061622
4244771461
4258065248
4267461418
4282530309
4204756249
4240915365
4048228635
4261493690
4278553667
4282834220
4274643866
4295334909
4274641859
4259095678
4293772062
4193362955
4292567427
4190532210
4189885322
4294838113
4273381368
4273385083
4273386060
4263992402
4273384149
4291154278
4295167685
4281093212
4273379435
4273379444
4273386061
4273381377
4273382291
4287397373
4281091343
4273380382
4273378985
4267225737
4255398471
4294396741
4273381371
4273385077

All extracted Job IDs: ['4290796891', '4259114971', '4220839450', '3849501886', '3627622884', '4292967130', '4260012242', '4255094324', '4259098167', '4259116372', '4048230355', '4276958556', '4273221487', '4240914500', '4281485208', '4275350664', '4168111890', '4294061622', '4244771461', '4258065248', '4267461418', '4282530309', '420475

In [13]:
job_list = []

for job_id in job_id_list:
    job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"

    try:
        job_response = requests.get(job_url)
        job_response.raise_for_status()
        job_soup = BeautifulSoup(job_response.text, "lxml")
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch job posting {job_id}: {e}")
        continue

    job_post = {
        "time_posted": None,
        "company_name": None,
        "job_title": None,
        "num_applicants": None,
        "job_id": None
    }

    job_post["job_id"] = job_id

    try:
        title_element = job_soup.select_one("h2.topcard__title")
        if title_element:
            job_post["job_title"] = title_element.get_text(strip=True)
    except Exception as e:
        print(f"Error extracting job title for {job_id}: {e}")

    try:
        company_element = job_soup.select_one("a.topcard__org-name-link")
        if company_element:
            job_post["company_name"] = company_element.get_text(strip=True)
    except Exception as e:
        print(f"Error extracting company name for {job_id}: {e}")

    try:
        time_element = job_soup.select_one("span.posted-time-ago__text")
        if time_element:
            job_post["time_posted"] = time_element.get_text(strip=True)
    except Exception as e:
        print(f"Error extracting posted time for {job_id}: {e}")

    try:
        applicants_selectors = [
            'figcaption.num-applicants__caption',
            'span.num-applicants__caption',
            'span.posted-time-ago__text'
        ]

        for selector in applicants_selectors:
            applicants_element = job_soup.select_one(selector)
            if applicants_element:
                text = applicants_element.get_text(strip=True)
                if "applicant" in text.lower():
                    job_post["num_applicants"] = text
                    break
    except Exception as e:
        print(f"Error extracting num applicants for {job_id}: {e}")

    job_list.append(job_post)
    print(f"Successfully scraped job {job_id}")

Successfully scraped job 4290796891
Successfully scraped job 4259114971
Successfully scraped job 4220839450
Successfully scraped job 3849501886
Successfully scraped job 3627622884
Successfully scraped job 4292967130
Successfully scraped job 4260012242
Successfully scraped job 4255094324
Successfully scraped job 4259098167
Successfully scraped job 4259116372
Successfully scraped job 4048230355
Successfully scraped job 4276958556
Successfully scraped job 4273221487
Successfully scraped job 4240914500
Successfully scraped job 4281485208
Successfully scraped job 4275350664
Successfully scraped job 4168111890
Successfully scraped job 4294061622
Successfully scraped job 4244771461
Successfully scraped job 4258065248
Successfully scraped job 4267461418
Successfully scraped job 4282530309
Successfully scraped job 4204756249
Successfully scraped job 4240915365
Successfully scraped job 4048228635
Failed to fetch job posting 4261493690: 429 Client Error: Request denied for url: https://www.linked

In [14]:
job_list

[{'time_posted': '6 days ago',
  'company_name': 'Ventura',
  'job_title': 'Full Stack Developer',
  'num_applicants': 'Over 200 applicants',
  'job_id': '4290796891'},
 {'time_posted': '2 months ago',
  'company_name': 'Fynd',
  'job_title': 'SDE - 1 | Fullstack Developer',
  'num_applicants': 'Over 200 applicants',
  'job_id': '4259114971'},
 {'time_posted': '4 months ago',
  'company_name': 'Softlink Global',
  'job_title': 'Fullstack Developer (0-3 years)',
  'num_applicants': 'Over 200 applicants',
  'job_id': '4220839450'},
 {'time_posted': '1 year ago',
  'company_name': 'Dimensionless Technologies',
  'job_title': 'Fullstack / Mern stack Developer',
  'num_applicants': 'Be among the first 25 applicants',
  'job_id': '3849501886'},
 {'time_posted': '2 years ago',
  'company_name': 'UnFound',
  'job_title': 'Fullstack Developer',
  'num_applicants': 'Be among the first 25 applicants',
  'job_id': '3627622884'},
 {'time_posted': '5 days ago',
  'company_name': 'AuxoAI',
  'job_tit

In [15]:
jobs_df = pd.DataFrame(job_list)
jobs_df

,time_posted,company_name,job_title,num_applicants,job_id
0,6 days ago,Ventura,Full Stack Developer,Over 200 applicants,4290796891
1,2 months ago,Fynd,SDE - 1 | Fullstack Developer,Over 200 applicants,4259114971
2,4 months ago,Softlink Global,Fullstack Developer (0-3 years),Over 200 applicants,4220839450
3,1 year ago,Dimensionless Technologies,Fullstack / Mern stack Developer,Be among the first 25 applicants,3849501886
4,2 years ago,UnFound,Fullstack Developer,Be among the first 25 applicants,3627622884
5,5 days ago,AuxoAI,Fullstack Engineer,182 applicants,4292967130
6,2 months ago,Talent Worx,Fullstack Developer,Be among the first 25 applicants,4260012242
7,2 months ago,Fynd,Fullstack Developer | Gauze,97 applicants,4255094324
8,2 months ago,Workduck,SDE - II (Full Stack Developer),136 applicants,4259098167
9,2 months ago,Fynd,SDE - 2 | Fullstack Developer,146 applicants,4259116372
